<span style="font-family:Papyrus; font-size:3em;">Homework 1</span>

<span style="font-family:Papyrus; font-size:2em;">Design of Experiments for Parameter Interactions</span>

# Problem

This homework is an elaboration of the analysis of lecture 6 on computational experiments using a two factor at a time design.

1. (6 pt) Revise the codes from lecture 6 (in common>util_doe6.py) to cacluate the interaction terms $\gamma$
in the model $y = \alpha_{1,i_1} + \alpha_{2, i_2} + \gamma_{1, 2, i_1, i_2}$. 

1. (6 pt) Analyze interactions for each floating species the first 5 parameters: J0_inputFlux, J1_k1, J1_Ki, J1_n, J2_k. 
   Your analysis should discuss the characteristics of the $\gamma$ for different combinations of parameters
   for both frequency and amplitude. Justify your choice of levels (percent changes) in parameter values.

1. (3 pt) Do you think that other parameters should be included in the study of interactions? Why?

# Programming Preliminaries
This section provides the setup to run your python codes.

In [1]:
IS_COLAB = False

In [2]:
# Code installs. Only needed once.
if IS_COLAB:
    !pip install matplotlib
    !pip install numpy
    !pip install pandas
    !pip install scipy
    !pip install tellurium
    !pip install seaborn

In [6]:
# Constants for standalone notebook
if not IS_COLAB:
    DIR = "/home/ubuntu/advancing-biomedical-models/common"  # Insert your local directory path
else:
    from google.colab import drive
    drive.mount('/content/drive')
    DIR = "/content/drive/MyDrive/Modeling_Class/Winter 2021/common"
import sys
sys.path.insert(0, DIR)

In [4]:
import wolf_model as wm
import util_doe1  # Various functions from lecture 5
import util_doe2  # Various functions from lecture 6

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
from scipy import fftpack
import tellurium as te
import seaborn as sns

# Calculate interaction terms ($\gamma$)
This section contains revised and/or new functions that calculate the interaction terms and plot the results.

In [11]:
def runExperimentsTwoParameters(parameter1, parameter2, percent1s, percent2s, isRelative=True):
    """
    Runs experiments for changes in multipleparameters of the model at different percent changes 
    in the parameter value (levels). Responses are the interaction terms gamma.
    
    Parameter
    ---------
    parameter1: str
    parameter2: str
    percent1s: list-int
    percent2s: list-int
    isRelative: bool
        True: values are percent changes relative to baseline
        False: absolute value
        
    Returns
    -------
    frequencyDF: DataFrame, amplitude: DataFrame
       index: tuple of percent changes in parameter values
       columns: molecule
       value: interaction term gamma
       index.name: string of tuple (rowParameter, columnParameter)
    """
    INDEX1 = "index1"
    INDEX2 = "index2"
    # Calculate the baseline values
    baseFrequencySer, baseAmplitudeSer = util_doe1.runExperiment({})
    #
    def calc(ser, isFrequency=True):
        if not isRelative:
            return ser
        if isFrequency:
            baseSer = baseFrequencySer
        else:
            baseSer = baseAmplitudeSer
        resultSer = 100*(ser - baseSer)/baseSer
        return resultSer
    #
    def makeSer(percent1, percent2, isFrequency):
        """
        Calculates the response for the percent changes
        """
        parameterDct = {parameter1: percent1, parameter2: percent2}
        frequencySer, amplitudeSer = util_doe1.runExperiment(parameterDct)
        if isFrequency:
            ser = frequencySer
        else:
            ser = amplitudeSer
        adjSer = calc(ser, isFrequency=isFrequency)
        return adjSer
    #
    def iterateLevels(isFrequency=None):
        if isFrequency is None:
            raise ValueError("Must specify isFrequency!")
        sers = []  # Collection of experiment results
        index1 = []
        index2 = []
        for percent1 in percent1s:
            for percent2 in percent2s:
                index1.append(percent1)
                index2.append(percent2)
                only1Ser = makeSer(percent1, 0, isFrequency)
                only2Ser = makeSer(0, percent2, isFrequency)
                bothSer = makeSer(percent1, percent2, isFrequency)
                interactionSer = bothSer - only1Ser - only2Ser
                sers.append(pd.DataFrame(interactionSer).transpose())
        resultDF = pd.concat(sers)
        resultDF[INDEX1] = index1
        resultDF[INDEX2] = index2
        resultDF = resultDF.set_index([index1, index2])
        for column in [INDEX1, INDEX2]:
            del resultDF[column]
        resultDF.index.name = str((parameter1, parameter2))
        return resultDF
    #
    frequencyDF = iterateLevels(isFrequency=True)
    amplitudeDF = iterateLevels(isFrequency=False)
    return frequencyDF, amplitudeDF

# Tests
percents = [ -2, 0, 2]
fDF, aDF = runExperimentsTwoParameters("J1_Ki", "J1_k1", percents, percents)

In [13]:
fDF

Glucose  fructose_1_6_bisphosphate  glyceraldehyde_3_phosphate  \
-2 -2  0.000000                   0.000000                    0.000000   
    0  0.000000                   0.000000                    0.000000   
    2  0.000000                   0.000000                    0.000000   
 0 -2  0.000000                   0.000000                    0.000000   
    0  0.000000                   0.000000                    0.000000   
    2  0.000000                   0.000000                    0.000000   
 2 -2 -5.882353                  -5.882353                   -5.882353   
    0  0.000000                   0.000000                    0.000000   
    2  0.000000                   0.000000                    0.000000   

       glycerate_3_phosphate  pyruvate  Acetyladehyde  External_acetaldehyde  \
-2 -2               0.000000  0.000000       0.000000               0.000000   
    0               0.000000  0.000000       0.000000               0.000000   
    2               0.000000  0.000000       0.000000               0.000000   
 0 -2               0.000000  0.000000       0.000000               0.000000   
    0               0.000000  0.000000       0.000000               0.000000   
    2               0.000000  0.000000       0.000000               0.000000   
 2 -2              -5.882353 -5.882353      -5.882353              -5.882353   
    0               0.000000  0.000000       0.000000               0.000000   
    2               0.000000  0.000000       0.000000               0.000000   

            ATP       ADP       NAD      NADH  
-2 -2  0.000000  0.000000  0.000000  0.000000  
    0  0.000000  0.000000  0.000000  0.000000  
    2  0.000000  0.000000  0.000000  0.000000  
 0 -2  0.000000  0.000000  0.000000  0.000000  
    0  0.000000  0.000000  0.000000  0.000000  
    2  0.000000  0.000000  0.000000  0.000000  
 2 -2 -5.882353 -5.882353 -5.882353 -5.882353  
    0  0.000000  0.000000  0.000000  0.000000  
    2  0.000000  0.000000  0.000000  0.000000

# Analyze the interactions.

You want to use ``util_doe2.runStudy`` to analyze interactions, but you want it to use the version of ``runExperimentsTwoParameters`` that you constructed in the previous section.
You can do this by taking advantage of the keyword argument ``func`` in ``runStudy``. You can see how to use this by looking at the tests at the bottom of the file ``util_doe2.py``.

# Discuss additional analyses that should be included.